# Testcase Data Retrieval for Earthquake Swarms in the PNSN Network
This notebook provides a worked example of data retrieval for the following earthquake swarm within the PNSN network:  
The May 2017 earthquake swarm near Bremerton  
        - **Note** this example uses just 2 days of the swarm: May 10-11, 2017.  
        - Catalog searches show the swarm can broadly be defined between May 8 and November 2017  
        - ...potentially longer - a declustering analysis would better define this.  
![Event map of earthquakes in the seismic swarm near Bremerton, WA](data/2017-05-10_BRM/image.png)  


- "Earthquake swarm NE of Bremerton" blog post by Renate Hartog on May 11, 2017  
https://www.pnsn.org/blog/2017/05/11/earthquake-swarm-ne-of-bremerton


### Script Author
Nathan Stevens: ntsteven@uw.edu

In [10]:
# Import dependencies
from obspy import UTCDateTime
from obspy.clients.fdsn import Client
import pandas as pd
import matplotlib.pyplot as plt
import os


# Define small helper function - I know it's somewhere in ObsPy, but this was easier than hunting it down...
def m2deg(x):
    """
    Convert distance `x` from meters to great-circle degrees on Earth
    """
    return x/111139.

In [11]:
# Initialize Client
client = Client('IRIS')

### Data selection
Use the following fields to define an event and station query kwargs for use with the `obspy`
 - **starttime**: [obspy.UTCDateTime] Start time of query in UTC
 - **endtime**:      [obspy.UTCDateTime] End time of query in UTC
 - **latitude**:     [float] centroid latitude for queries in *degrees North*
 - **longitude**:    [float] centroid longitude for queries in *degrees East*
 - **maxradius**: [float] maximum radius for query in *degrees*

In [16]:
# Define fields for query
fields = ('starttime','endtime','latitude','longitude','maxradius')
TSstr = "2017-05-11T00:00:00"
TEstr = "2017-05-12T23:59:59"
# Define data selection kwargs for events from IRIS catalog
qevent_kwargs = dict(zip(fields,[UTCDateTime(TSstr),\
                                UTCDateTime(TEstr),\
                                47.5828,-122.57841,m2deg(7.5e3)]))
# Define data selection kwargs for stations in IRIS catalog
qstation_kwargs = dict(zip(fields,[UTCDateTime(TSstr),\
                                   UTCDateTime(TEstr),\
                                47.5828,-122.57841,m2deg(20e3)]))

# Limit channels to those coming from broadbands ?H? and accelerometers ?N?
CLIST = 'HHZ,HHN,HH1,HHE,HH2,ENZ,EN1,ENN,EN2,ENE,BHZ,BHN,BHZ'

In [17]:
# Download inventory with specified query down to channel 
# (ML generally operates on data w/o instrument response correction)
inv = client.get_stations(**qstation_kwargs,level='channel',channel=CLIST)#,channel='??[ZNE12]')
print(inv)

Inventory created at 2023-10-05T23:38:36.657900Z
	Created by: IRIS WEB SERVICE: fdsnws-station | version: 1.1.52
		    http://service.iris.edu/fdsnws/station/1/query?starttime=2017-05-...
	Sending institution: IRIS-DMC (IRIS-DMC)
	Contains:
		Networks (1):
			UW
		Stations (22):
			UW.BABE (Bainbridge, WA, USA)
			UW.BES3 (Bremerton Swarm Portable 3, WA, USA)
			UW.BES4 (Bremerton Swarm Portable 4, WA, USA)
			UW.BES5 (Bremerton Swarm Portable 5, WA, USA)
			UW.GMW (Gold Mountain, WA, USA)
			UW.GNW (Green Mountain, WA, USA)
			UW.GTWN (Georgetown Playfield ANSS-SMO)
			UW.HART (Harbor Island, Seattle, WA, USA)
			UW.HOLY (Holy Rosary ANSS-SMO)
			UW.KCAM (King County Airport Maintenance Shop, Seattle, WA, USA)
			UW.KDK (Kingdome, Seattle, WA, USA)
			UW.KIMR (Kitsap County HHW Collection Facility, Bremerton, WA, USA)
			UW.KINR (North Road Shop, Poulsbo, WA, USA)
			UW.KITP (Kitsap Treatment Plant, WA, USA)
			UW.LAWT (Lawton, Seattle, WA, USA)
			UW.MNWA (Manchester, WA)
			UW.PSNS 

In [ ]:
# Download catalog from Client
cat = client.get_events(**qevent_kwargs)
print(cat.__str__(print_all=True))

In [ ]:
# Load PNSN queried catalog data from custom queries for Bremerton-Adjacent Swarm (BRM)
df_BRM_full = pd.read_csv(os.path.join('data','2017-05-10_BRM','pnsn_event_catalog_2017_BRM_cluster.csv'),\
                          parse_dates=['Time UTC'],index_col='Time UTC')
# Sort by index
df_BRM_full = df_BRM_full.sort_index()
# Filter by query times
# Translate from obspy.UTCDateTime --> pandas.Timestamp
pdTS = pd.Timestamp(TSstr)
pdTE = pd.Timestamp(TEstr)

# Filter to period of interest
df_EVE = df_BRM_full[(df_BRM_full.index >= pdTS) & (df_BRM_full.index <= pdTE)]
print(len(df_EVE))
print(df_EVE)


In [ ]:
# Use inventory to pull data for the period of interest and save to disk in 1 hour chunks
ts = UTCDateTime(TSstr)
te = ts + 3600.
save_dir = os.path.join('data','2017-05_BRM')
while ts < UTCDateTime(TEstr):
    st = client.get_waveforms(starttime=ts,endtime=te)
    ts += 3600.
    te += 3600.


In [ ]:
# Use IRIS webservices / FDSN to query station, event, and waveform data 


In [ ]:
# # Plot the catalog
# fig = plt.figure(figsize=(10,10))
# axa = fig.add_subplot(211)
# axa.plot(df_BRM_full['Magnitude'],'.')
# axa.set_xlabel('UTC Date Time')
# axa.set_ylabel('Magnitude (mixed methods)')
# axb = axa.twinx()
# axb.plot(df_BRM_full['Magnitude'].cumsum(),'r-',alpha=0.5)
# axb.set_ylabel('Cumulative magnitude',labelpad=15)

# # # Define window to assess
# # TS = pd.Timestamp("2017-05-11")
# # TE = pd.Timestamp("2017-05-13T00:00:00")

# axa.fill_between([TS,TE],[axa.get_ylim()[0]]*2,[axa.get_ylim()[1]]*2,color='yellow')


# axc = fig.add_subplot(212)
# IND = (df_BRM_full.index >= TS) & (df_BRM_full.index <= TE)
# axc.plot(df_BRM_full[IND]['Magnitude'],'.')
# axd = axc.twinx()
# axd.plot((10**df_BRM_full[IND]['Magnitude']).cumsum(),'r-',alpha=0.5)

# # axc = fig.add_subplot(223)
# # ch = axc.scatter(df_BRM_full['Lon'],df_BRM_full['Lat'],s = 5**df_BRM_full['Magnitude'],c=df_BRM_full.index)
# # plt.colorbar(ax=axc,cax=ch)